In [ ]:
import pandas as pd
import numpy as np
import pickle

In [ ]:
df0 = pd.read_csv("../grab-ai-safety-data/features/part-00000-e6120af0-10c2-4248-97c4-81baf4304e5c-c000.csv")
df1 = pd.read_csv("../grab-ai-safety-data/features/part-00001-e6120af0-10c2-4248-97c4-81baf4304e5c-c000.csv")
df2 = pd.read_csv("../grab-ai-safety-data/features/part-00002-e6120af0-10c2-4248-97c4-81baf4304e5c-c000.csv")
df3 = pd.read_csv("../grab-ai-safety-data/features/part-00003-e6120af0-10c2-4248-97c4-81baf4304e5c-c000.csv")
df4 = pd.read_csv("../grab-ai-safety-data/features/part-00004-e6120af0-10c2-4248-97c4-81baf4304e5c-c000.csv")
df5 = pd.read_csv("../grab-ai-safety-data/features/part-00005-e6120af0-10c2-4248-97c4-81baf4304e5c-c000.csv")
df6 = pd.read_csv("../grab-ai-safety-data/features/part-00006-e6120af0-10c2-4248-97c4-81baf4304e5c-c000.csv")
df7 = pd.read_csv("../grab-ai-safety-data/features/part-00007-e6120af0-10c2-4248-97c4-81baf4304e5c-c000.csv")
df8 = pd.read_csv("../grab-ai-safety-data/features/part-00008-e6120af0-10c2-4248-97c4-81baf4304e5c-c000.csv")
df9 = pd.read_csv("../grab-ai-safety-data/features/part-00009-e6120af0-10c2-4248-97c4-81baf4304e5c-c000.csv")
response = pd.read_csv("../grab-ai-safety-data/labels/part-00000-e9445087-aa0a-433b-a7f6-7f4c19d78ad6-c000.csv")

In [ ]:
df_features = pd.concat([df1, df2, df3, df4, df5, df6, df7, df8, df9], axis=0)

In [ ]:
df = pd.merge(
    df_features,
    response,
    how="left",
    on="bookingID"
)

In [ ]:
# FIrst note that second itself as a feature is totally garbage. But can be used to calculate distance of a trip?
# Do we care about the x y z in acceleration and gyro? or just take the max?
# Does accuracy and bearing even matter?

## Feature Extraction

In [ ]:
def weighted_sum(x, y):
    return (x * y).sum()

In [ ]:
df["max_acceleration"] = df[["acceleration_x", "acceleration_y", "acceleration_z"]].max(axis=1)
df["max_gyro"] = df[["gyro_x", "gyro_y", "gyro_z"]].max(axis=1)
df["Speed_abs"] = np.abs(df["Speed"])

In [ ]:
df.head()

In [ ]:
df_sorted = df.sort_values(["bookingID", "second"])
df_sorted["time_diffs"] = df_sorted["second"].diff()
df_sorted["Bearing_diffs"] = df_sorted["Bearing"].diff()
mask = df_sorted["bookingID"] != df_sorted["bookingID"].shift(1)
df_sorted["time_diffs"][mask] = 0
df_sorted["Bearing_diffs"][mask] = 0
df_sorted["distance_covered"] = df_sorted["Speed_abs"] * df_sorted["time_diffs"]
df_sorted["Bearing_diffs"] = np.abs(df_sorted["Bearing_diffs"])

In [ ]:
df_sorted.head(100)

In [ ]:
df_sorted["std_speed"] = df_sorted["Speed"]
df_sorted["std_acc"] = df_sorted["max_acceleration"]
df_sorted["std_gyro"] = df_sorted["max_gyro"]
df_sorted["std_bearing"] = df_sorted["Bearing_diffs"]

In [ ]:
df_grouped = df_sorted.groupby("bookingID").agg({
    "Speed": "mean",
    "max_acceleration": "max",
    "max_gyro": "max",
    #"Bearing_diffs": "mean",
    "distance_covered": "sum",
    "std_speed": "std",
    "std_acc": "std",
   # "std_gyro": "std",
    "std_bearing": "std",
    "label": "max"
})

In [ ]:
with open('outputs/df_cleaned.pickle', 'wb') as f:
    pickle.dump(df_grouped, f)